In [1]:

import pandas as pd
import numpy as np
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [3]:

column_names = [
    "subject#", "age", "sex", "test_time", "motor_UPDRS", "total_UPDRS",
    "Jitter(%)", "Jitter(Abs)", "Jitter:RAP", "Jitter:PPQ", "Jitter:DDP",
    "Shimmer", "Shimmer(dB)", "Shimmer:APQ3", "Shimmer:APQ5",
    "Shimmer:APQ", "Shimmer:DDA", "NHR", "HNR"
]
df = pd.read_csv("parkinsons_updrs.data.csv", names=column_names, header=None)
df.columns = df.columns.str.strip()
df = df[~df.apply(lambda row: any(row.astype(str).str.contains("test_time", case=False)), axis=1)]
df = df.dropna()
df = df.astype(float)


In [5]:

features = [col for col in df.columns if col not in ['subject#', 'motor_UPDRS', 'total_UPDRS']]
groups = df['subject#']
gkf = GroupKFold(n_splits=5)
for train_idx, test_idx in gkf.split(df[features], df['total_UPDRS'], groups):
    break
X_train = df.iloc[train_idx][features]
X_test = df.iloc[test_idx][features]
y_train_motor = df.iloc[train_idx]['motor_UPDRS']
y_test_motor = df.iloc[test_idx]['motor_UPDRS']
y_train_total = df.iloc[train_idx]['total_UPDRS']
y_test_total = df.iloc[test_idx]['total_UPDRS']

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [7]:

model_motor = RandomForestRegressor(n_estimators=200, max_depth=20, random_state=42)
model_motor.fit(X_train_scaled, y_train_motor)

model_total = RandomForestRegressor(n_estimators=200, max_depth=20, random_state=42)
model_total.fit(X_train_scaled, y_train_total)


RandomForestRegressor(max_depth=20, n_estimators=200, random_state=42)

In [8]:

def interpret_updrs_en(total_updrs):
    if total_updrs < 15:
        return "✅ Mild symptoms – possibly early-stage Parkinson's."
    elif total_updrs < 30:
        return "⚠️ Moderate Parkinson's – follow-up and treatment recommended."
    else:
        return "❌ Advanced Parkinson's – urgent clinical intervention may be needed."

sample_values = []
print("Please enter the following 16 features for the new patient:")

input_features = ["age", "sex", "test_time", "Jitter(%)", "Jitter(Abs)",
                  "Jitter:RAP", "Jitter:PPQ", "Jitter:DDP", "Shimmer",
                  "Shimmer(dB)", "Shimmer:APQ3", "Shimmer:APQ5", "Shimmer:APQ",
                  "Shimmer:DDA", "NHR", "HNR"]

for feature in input_features:
    while True:
        try:
            val = float(input(f"Enter {feature}: "))
            sample_values.append(val)
            break
        except ValueError:
            print("Invalid input. Please enter a number.")


Please enter the following 16 features for the new patient:


Enter age:  68
Enter sex:  1
Enter test_time:  12.5
Enter Jitter(%):  0.0051
Enter Jitter(Abs):  0.00004
Enter Jitter:RAP:  0.0021
Enter Jitter:PPQ:  0.0029
Enter Jitter:DDP:  0.0010
Enter Shimmer:  0.023
Enter Shimmer(dB):  0.16
Enter Shimmer:APQ3:  0.0041
Enter Shimmer:APQ5:  0.0039
Enter Shimmer:APQ:  0.0044
Enter Shimmer:DDA:  0.0031
Enter NHR:  0.0025
Enter HNR:  22.0


In [9]:

sample = np.array(sample_values).reshape(1, -1)
sample_scaled = scaler.transform(sample)
motor_pred = model_motor.predict(sample_scaled)[0]
total_pred = model_total.predict(sample_scaled)[0]

motor_min, motor_max = 0, 50
total_min, total_max = 0, 60
motor_pred_denorm = motor_pred * (motor_max - motor_min) + motor_min
total_pred_denorm = total_pred * (total_max - total_min) + total_min

def interpret_updrs_real(total_updrs):
    if total_updrs < 15:
        return "✅ Mild symptoms – possibly early-stage Parkinson's."
    elif total_updrs < 30:
        return "⚠️ Moderate Parkinson's – follow-up and treatment recommended."
    else:
        return "❌ Advanced Parkinson's – urgent clinical intervention may be needed."

print("\n📊 Prediction Results (Raw):")
print(f"Predicted Motor UPDRS: {motor_pred:.5f}")
print(f"Predicted Total UPDRS: {total_pred:.5f}")

print("\n📊 Adjusted Prediction Results (Real Scale):")
print(f"🧮 Adjusted Motor UPDRS: {motor_pred_denorm:.2f}")
print(f"🧮 Adjusted Total UPDRS: {total_pred_denorm:.2f}")
print(f"Clinical Interpretation: {interpret_updrs_real(total_pred_denorm)}")

print("\n📌 Note: These scores are scaled back from normalized data.")



📊 Prediction Results (Raw):
Predicted Motor UPDRS: 0.00010
Predicted Total UPDRS: 0.00044

📊 Adjusted Prediction Results (Real Scale):
🧮 Adjusted Motor UPDRS: 0.00
🧮 Adjusted Total UPDRS: 0.03
Clinical Interpretation: ✅ Mild symptoms – possibly early-stage Parkinson's.

📌 Note: These scores are scaled back from normalized data.


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
